In [19]:
def crime_cleaner(data):
    # Merge key with crime db to gain state / city names
    washing = data.merge(key00,left_on=['FIPS_CTY','FIPS_ST'], right_on=['County FIPS Code','State FIPS Code'])
    # Rearrange columns
    washing = washing[['STUDYNO','EDITION','PART','IDNO','FIPS_ST','State FIPS Code','State Abbreviation','FIPS_CTY','County FIPS Code','GU Name','CPOPARST','AG_ARRST','JURFLAG','COVIND', 'GRNDTOT','P1TOT','P1VLNT','P1PRPTY','MURDER','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY']]
    
    # Select only wanted columns
    clean = washing[['State Abbreviation','GU Name','MURDER','P1TOT','P1VLNT','P1PRPTY','RAPE','ROBBERY','AGASSLT','BURGLRY','LARCENY','MVTHEFT','ARSON','OTHASLT','FRGYCNT','FRAUD','EMBEZL','STLNPRP','VANDLSM','WEAPONS','COMVICE','SEXOFF','DRUGTOT','DRGSALE','COCSALE','MJSALE','SYNSALE','OTHSALE','DRGPOSS','COCPOSS','MJPOSS','SYNPOSS','OTHPOSS','GAMBLE','BOOKMKG','NUMBERS','OTGAMBL','OFAGFAM','DUI','LIQUOR','DRUNK','DISORDR','VAGRANT','ALLOTHR','SUSPICN','CURFEW','RUNAWAY']]
    return clean
def wage_cleaner(data):
        # Drop Unwanted columns
    base = data.drop(['Area\nCode','Own','NAICS','Qtr','Status Code'],axis=1)
    # Select County level data
    washing = base.loc[(base['Area Type'] == 'County')].copy()
    washing['Cnty'] = washing['Cnty'].astype('int64') # Convert to int for merging // Throws error
    washing['St'] = washing['St'].astype('int64')    # Convert to into for merging // Throws error
    # Merge with key
    washing = washing.merge(key00,left_on=['St','Cnty'], right_on=['State FIPS Code','County FIPS Code'])
    # Rearrange columns and drop unwanted columns
    county_wages = washing[['State Abbreviation','GU Name', 'Year', 'Area Type', 'St Name', 'Area', 'Ownership','Industry', 'Establishment Count', 'July Employment','August Employment', 'September Employment', 'Total Quarterly Wages','Average Weekly Wage', 'Employment Location Quotient Relative to U.S.','Total Wage Location Quotient Relative to U.S.']]
    county_wages['Establishment Count'] = county_wages['Establishment Count'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['Establishment Count'] = pd.to_numeric(county_wages['Establishment Count'])   # Convert to interger // Throws Error
    county_wages['September Employment'] = county_wages['September Employment'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['September Employment'] = pd.to_numeric(county_wages['September Employment'])    # Convert to interger // Throws Error  
    county_wages['Average Weekly Wage'] = county_wages['Average Weekly Wage'].str.replace(',','')  # Get rid of comma // Throws Error
    county_wages['Average Weekly Wage'] = pd.to_numeric(county_wages['Average Weekly Wage'])  # Convert to interger // Throws Error
    county_wages['Industry'] = county_wages['Industry'].str.replace('0','') # Formatting to fix year 2018 industry codes
    county_wages['Industry'] = county_wages['Industry'].str.replace('1','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('2','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('3','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('4','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('5','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('6','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('7','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('8','')
    county_wages['Industry'] = county_wages['Industry'].str.replace('9','')
    
#     state_wages = base.loc[(base['Area Type'] == 'State')]   # State Level Wages
#     nation_wages = base.loc[(base['Area Type'] == 'Nation')]    # Nation Level Data
#     urban_wages = base.loc[(base['Area Type'] == 'MSA')]   # Metropolitan Level Data
    return county_wages #,state_wages,nation_wages,urban_wages

In [3]:
# test new search function for large sample databases
def test_search(county,state):
    db09 = db2009.loc[(db2009['State Abbreviation'] == state) & (db2009['GU Name'] == county)]
    db10 = db2010.loc[(db2010['State Abbreviation'] == state) & (db2010['GU Name'] == county)]
    db12 = db2012.loc[(db2012['State Abbreviation'] == state) & (db2012['GU Name'] == county)]
    db14 = db2014.loc[(db2014['State Abbreviation'] == state) & (db2014['GU Name'] == county)]
    db16 = db2016.loc[(db2016['State Abbreviation'] == state) & (db2016['GU Name'] == county)]
    db18 = economic18.loc[(economic18['State Abbreviation'] == state) & (economic18['GU Name'] == county)]
    return pd.concat([db09,db10,db12,db14,db16])

def test_func(new_world,keys,s,n):
    bin = []
    x = new_world.loc[new_world['Industry'] == n]
    x = x[s].tolist()
    for a in x:
        bin.append(a)
    return bin
def test_funk(data,crime):
    y = [2009,2010,2012,2014,2016]
    bin = []
    for a in keys:
        for b in y:
            x = data.loc[(data['Year'] == b) & (data['GU Name'] == a[4:]) & (data['State Abbreviation'] == a[:2])][:1][crime].tolist()
            for c in x:
                bin.append(c)
    return bin
def big_industry_bot(samp_db):
        # Select most common industries from sample
    trash = []
    industry_all = []  # List of all unique industries in sample
    for a in keys:
        for b in samp_db[a]['Industry']:
            trash.append(b)
    for a in trash:
        if a not in industry_all:
            industry_all.append(a)
    base_dic = {}
    for a in industry_all:
        base_dic[a] = [trash.count(a)]

    base99 = pd.DataFrame(data=base_dic)
    base99 = base99.transpose()
    base99 = base99.nlargest(6,0)
    base99 = base99.transpose()
    industry = base99.columns.tolist()
    industry = industry[1:] # Remove 'all industries' from count
    return industry

In [1]:
# Called from bot 97
def bot99(data):
    x = data.dtypes
    x = pd.DataFrame(data=x)
    x = x.transpose()
    # Create list of column names that are not int64
    target = []
    for a in x:
        if x[a][0] == 'object':
            b = str(x[a])[:-15]
            b = b[18:]
            target.append(b)
    # Convert string columns to interger columns
    for a in target:
        data[a] = data[a].str.replace(',','')
        data[a] = data[a].astype(float)
    return data

def bot98(data,industry):
    # Create a list of all industry data entrys
    l1 = []
    for a in data['Industry']:
        l1.append(a)
    print('Count of target industries in sample \n\n MUST ALL BE SAME\n')
    # Find count of each indutry in sample
    l2 = []
    for a in industry:
        #print(a,'//',l1.count(a)) # Print out count of each industry in data frame
        l2.append(l1.count(a))

    c = l2[0]
    for a in l2[1:]:
        if a == c:
            print('Check')
        else:
            print('Failed Industry Count Check')

In [2]:
def bot97(data):
    data_a = bot99(data[y])
    return pd.concat([data[z[:8]],data_a],1)

def bot96(edata,cdata,cols):
#     db2009 = crime09.merge(economic09,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
#     db2009 = db2009[cols]
    db_a = cdata.merge(edata,left_on=['State Abbreviation','GU Name'],right_on=['State Abbreviation','GU Name'])
    return db_a[cols]